# HOMEWORK 2
Corso di Sistemi informativi evoluti e big data

*Sabbadini Marco mat. 723421*


In [2]:
import influxdb_client
import pandas as pd
from datetime import datetime

/Users/marcosabbadini/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
org_name = "UNIBS"
bucket_name = "Homework2"
token = "5m11ynHOS5lK1Dp2BcSk9JP8sLxziwmIkn8EFp_2-Zp6XTOdZ54KZ4ppitszQAVE0OM0qLfY9Tt7o5jHDcEqCQ=="
url = "https://localhost:8086"

In [4]:
client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org_name
)

In [5]:
write_api = client.write_api()

## DATA UPLOADING

In [6]:
city_data = pd.read_csv('archive/city_attributes.csv')

In [7]:
len(city_data["City"])

36

COMMENTA

In [8]:
def create_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        "Autumn"

In [11]:
def humidity_warning(humidity):
    if humidity > 98:
        return "Alert"
    elif humidity > 90:
        return "Warning"
    else:
        "OK"
    

CARICAMENTO E SOGLIE ALERT

In [9]:
humidity_data = pd.read_csv('archive/humidity.csv')

In [10]:
humidity_data.head()

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN
1,2012-10-01 13:00:00,76.0,81.0,88.0,81.0,88.0,82.0,22.0,23.0,50.0,...,71.0,58.0,93.0,68.0,50.0,63.0,22.0,51.0,51.0,50.0
2,2012-10-01 14:00:00,76.0,80.0,87.0,80.0,88.0,81.0,21.0,23.0,49.0,...,70.0,57.0,91.0,68.0,51.0,62.0,22.0,51.0,51.0,50.0
3,2012-10-01 15:00:00,76.0,80.0,86.0,80.0,88.0,81.0,21.0,23.0,49.0,...,70.0,57.0,87.0,68.0,51.0,62.0,22.0,51.0,51.0,50.0
4,2012-10-01 16:00:00,77.0,80.0,85.0,79.0,88.0,81.0,21.0,23.0,49.0,...,69.0,57.0,84.0,68.0,52.0,62.0,22.0,51.0,51.0,50.0


In [12]:
SOGLIA_HUM_ALERT = 98.0
SOGLIA_HUM_WARNING = 90.0


In [13]:
def caricamento_dati_influxdb(humidity, city):
    
    punto = (
        influxdb_client.Point(city)
        .field("Humidity", humidity)
    )
    write_api.write(bucket=bucket_name, org=org_name, record=punto)

def control_humidity_alert(humidity):
    """Controlla se le letture dei sensori superano le soglie predefinite e genera un alert."""
    if humidity > SOGLIA_HUM_ALERT:
        print(f"ALERT: High Humidity Value: {humidity:.2f}% (Soglia: {SOGLIA_HUM_ALERT}%)")

    if humidity > SOGLIA_HUM_WARNING:
        print(f"WARNING: High Humidity Value: {humidity:.2f}% (Soglia: {SOGLIA_HUM_WARNING}%)")